<a href="https://colab.research.google.com/github/cqueiro2/ProjectOx/blob/main/ProjectOx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
!pip install -U -q Pillow

In [ ]:
!pip install -U -q google-cloud-vision google-cloud-language

In [ ]:
!pip install -U -q tensorflow

In [ ]:
!pip install sentence-transformers

#Identificação da imagens e qual sua similaridade
Combinando Embeddings de Imagem e Texto
usa um modelo de linguagem pré-treinado (Sentence Transformer) para gerar embeddings de texto e compara-os com embeddings de imagens extraídos usando o Inception V3:


In [322]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import PIL

# URL do modelo Inception V3 no TensorFlow Hub
module_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"

# Carregue o modelo
model = hub.load(module_url)
# Função para pré-processar a imagem

def preprocess_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [500, 400])
  image = tf.keras.applications.inception_v3.preprocess_input(image)
  return image

# Lista de caminhos para as imagens
image_paths = ["/home/img/bovino1.jpg", "/home/img/bovino2.jpg"]
  # Lista para armazenar os embeddings
embeddings = []
  # Itere sobre a lista de imagens
for image_path in image_paths:
    # Carregue e pré-processe a imagem
     image = preprocess_image(image_path)
      # Extraia o embedding
     embedding = model(image[tf.newaxis, ...])[0]
      # Adicione o embedding à lista
     embeddings.append(embedding)
    #Exemplo: calcule a similaridade cosseno entre o primeiro e o segundo embedding
similarity = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
print("Similaridade cosseno:", similarity)

Similaridade cosseno: 0.9280016


In [325]:
import google.generativeai as genai
from pathlib import Path
import hashlib
from IPython.display import Markdown
from google.colab import userdata
userdata.get('API_KEY')


GOOGLE_API_KEY=userdata.get('API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [326]:
# Set up the model
generation_config = {
  "candidate_count":1,
  "temperature": 0.5,

}

In [327]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [328]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [329]:
chat = model.start_chat(history=[])

Selecionar e informar a Raça e sua categorias

In [330]:
def selectText():
    raca = input("Especifique a raça do bovino: ")
    options = ["Categoria: Bezerro Idade (meses): 0-1, Peso Médio (Kg): 30,  Peso Macho: 32, Peso Fêmea: 28",
               "Categoria: Bezerro Idade (meses): 03,  Peso Médio (Kg): 60,  Peso Macho: 65, Peso Fêmea: 55",
               "Categoria: Bezerro Idade (meses): 05, Peso Médio (Kg): 90,  Peso Macho: 98, Peso Fêmea: 82",
               "Categoria: Bezerro Idade (meses): 07, Peso Médio (Kg): 120,  Peso Macho: 130, Peso Fêmea: 110",
               "Categoria: Bezerro Idade (meses): 09, Peso Médio (Kg): 150,  Peso Macho: 162, Peso Fêmea: 138",
               "Categoria: Bezerro Idade (meses): 11, Peso Médio (Kg): 180,  Peso Macho: 195, Peso Fêmea: 165",
                "Categoria: Garrote Idade (meses): 12, Peso Médio (Kg): 210,  Peso Macho: 230, Peso Fêmea: 190",
                "Categoria: Garrote Idade (meses): 13, Peso Médio (Kg): 240,  Peso Macho: 260, Peso Fêmea: 220",
                "Categoria: Garrote Idade (meses): 14, Peso Médio (Kg): 270,  Peso Macho: 290, Peso Fêmea: 250",
               "Categoria: Garrote Idade (meses): 15, Peso Médio (Kg): 300, Peso Macho: 320, Peso Fêmea: 280",
               "Categoria: Garrote Idade (meses): 16, Peso Médio (Kg): 330,  Peso Macho: 350, Peso Fêmea: 310",
               "Categoria: Garrote Idade (meses): 17, Peso Médio (Kg): 360,  Peso Macho: 380, Peso Fêmea: 340",
                "Categoria: Garrote Idade (meses): 18, Peso Médio (Kg): 390,  Peso Macho: 410, Peso Fêmea: 370",
               "Categoria: Adulto Idade (meses): 19+, Peso Médio (Kg): 420-550,  Peso Macho: 450-600, Peso Fêmea: 400-500"]

    # imprimir as opções
    for i, option in enumerate(options):
        print(f"{i+1}. {option}")

    # usuario seleciona
    while True:
        try:
            selection = int(input("Escolha numero da Categoria do animal (ou 0 to sair): "))
            if 0 <= selection <= len(options):
                break
            else:
                print("Seleção inválida. Por favor insira um número entre 1 e", len(options), "or 0 to exit.")
        except ValueError:
            print("Informação inválida. Please enter a number.")

    if selection != 0:
        selected_option = options[selection - 1]
        prompt=(f"Opção selecionada: Raça: {raca} {selected_option}")
    return prompt


In [331]:
def promptResult(prompt):
    prompt=f"{selectText()}"
    response = chat.send_message(prompt)
    return response.text



In [332]:
def promptImage():
  promptResult(prompt)
  prompt_parts = [
    "input: ",
    genai.upload_file("/home/img/bovino1.jpg"),
  ]

  return

Gerar o resultado da identificação informada com base na imagem.

In [334]:
if similarity <= 1.0:
    promptImage()
    response = model.generate_content(prompt_parts)
    print(response.text)
else:
   print("Imagem não possui similaridade")

Especifique a raça do bovino: Nelore
1. Categoria: Bezerro Idade (meses): 0-1, Peso Médio (Kg): 30,  Peso Macho: 32, Peso Fêmea: 28
2. Categoria: Bezerro Idade (meses): 03,  Peso Médio (Kg): 60,  Peso Macho: 65, Peso Fêmea: 55
3. Categoria: Bezerro Idade (meses): 05, Peso Médio (Kg): 90,  Peso Macho: 98, Peso Fêmea: 82
4. Categoria: Bezerro Idade (meses): 07, Peso Médio (Kg): 120,  Peso Macho: 130, Peso Fêmea: 110
5. Categoria: Bezerro Idade (meses): 09, Peso Médio (Kg): 150,  Peso Macho: 162, Peso Fêmea: 138
6. Categoria: Bezerro Idade (meses): 11, Peso Médio (Kg): 180,  Peso Macho: 195, Peso Fêmea: 165
7. Categoria: Garrote Idade (meses): 12, Peso Médio (Kg): 210,  Peso Macho: 230, Peso Fêmea: 190
8. Categoria: Garrote Idade (meses): 13, Peso Médio (Kg): 240,  Peso Macho: 260, Peso Fêmea: 220
9. Categoria: Garrote Idade (meses): 14, Peso Médio (Kg): 270,  Peso Macho: 290, Peso Fêmea: 250
10. Categoria: Garrote Idade (meses): 15, Peso Médio (Kg): 300, Peso Macho: 320, Peso Fêmea: 280
